PROJETO NLP
ANALISE DE SENTIMENTOS DE TWEETS

In [1]:
#pip install 

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from unidecode import unidecode
from nltk.stem.porter import *
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer 
stemmer = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [3]:
 df = pd.read_csv('Train3Classes.csv')
df.head(20)

,id,tweet_text,tweet_date,sentiment,query_used
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,Tue Oct 09 18:00:01 +0000 2018,2,folha
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,Sun Sep 30 04:11:28 +0000 2018,0,:(
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",Mon Sep 17 17:44:06 +0000 2018,2,exame
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,Tue Oct 02 01:37:06 +0000 2018,0,:(
4,1047326854229778432,@SiCaetano_ viva o caos :),Wed Oct 03 03:25:55 +0000 2018,1,:)
5,1046774830857695232,@ppolkiss Parabéns moço :D,Mon Oct 01 14:52:22 +0000 2018,1,:)
6,1032054856712945664,Trago verdades #fato https://t.co/DsbkrJgFVj,Wed Aug 22 00:00:27 +0000 2018,2,#fato
7,1046740081090981889,"A coleção aumentou, mas o espaço do quaro ""dim...",Mon Oct 01 12:34:17 +0000 2018,1,:)
8,1046768079349403648,@abreujaviera Amigaa :(,Mon Oct 01 14:25:32 +0000 2018,0,:(
9,1046760775182888961,fiquei exatos 45 minutos tentando conectar meu...,Mon Oct 01 13:56:31 +0000 2018,1,:)


In [4]:
df['query_used'==':)'] =1
df.head()

,id,tweet_text,tweet_date,sentiment,query_used,False
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,Tue Oct 09 18:00:01 +0000 2018,2,folha,1
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,Sun Sep 30 04:11:28 +0000 2018,0,:(,1
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",Mon Sep 17 17:44:06 +0000 2018,2,exame,1
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,Tue Oct 02 01:37:06 +0000 2018,0,:(,1
4,1047326854229778432,@SiCaetano_ viva o caos :),Wed Oct 03 03:25:55 +0000 2018,1,:),1


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [6]:
stopwords = set(stopwords.words('portuguese'))

list_sem_acento = []
for word in stopwords:
      word = unidecode(word)
      list_sem_acento.append(word)


In [8]:
# Pipeline - Text Preprocessing
def preprocessing(string):
    ###
    # Remove Numbers
    string = re.sub(r'\d', '', string)
    ###
    # Remove Special Characters
    string = re.sub(r"[^a-zA-Z0-9]+", ' ', string)
    ###
    # Lowercase words
    string = string.lower()
    ###
    # Word Tokenize
    tweet_tokenizer = TweetTokenize(string)
    ###
    # Remove Stopwords
    filtered_words = []
    for w in words:
        if w not in stopwords:
            filtered_words.append(w)
    ###
    # Stemming Words
    stem_words = []
    for w in filtered_words:
        s_words = stemmer.stem(w)
        stem_words.append(s_words)
    ###
    return stem_words

In [9]:
tweet_tokenizer = TweetTokenizer()
trainer = df[['tweet_text','sentiment']].sample(frac=0.25, replace=False)
train = trainer['tweet_text']
classe = trainer['sentiment']

In [10]:
clean_list = []

for word in train:
    if word not in list_sem_acento:
        word = word.lower()
        word= unidecode(word)
        word = re.sub(r"http\S+", "", word)
        clean_list.append(word)
        
print('A Lista sem stopwords é: ', clean_list)

A Lista sem stopwords é:  ["'daqui ate a data da eleicao muita coisa pode mudar', diz meirelles sobre ibope  #g1 ", 'nossa eu to sensivel demais com tudo o que ta acontecendo no mundo e na minha vida :(', '@ederetti @manonwhitebeak vergonha do meu pais regiao sul :(', 'elite e a reuniao de riverdale, la casa de papel, revenge e rebelde  amando :)', 'anda ca que eu trato disso :)) ', 'documentario observa vida de maria callas sob ponto de vista da soprano ', 'bora retribuir os unf :d #quintadetremurasdv', '@lierson mano do ceu, nem jantei hoje :(', '@kccgxlr ai mano :(((((((( deve ser pq ta sem memoria', 'o brasil e uma grande piada de mau gosto :(', 'goleiro bruno ja tem direito ao semiaberto e pode deixar prisao a qualquer momento, diz defesa  #g1 ', '@xgotahs eh uma das minhas bandinhas fav :)) tomara q goste', 'cansei de ser inutil :d', 'adicionei um video a uma lista de reproducao do @youtube  fortnite ! season 6 ! novo battle pass :d novo', '@luciano_hang vai que e tua sonegador :

In [11]:
vectorize = CountVectorizer(analyzer='word',tokenizer=tweet_tokenizer.tokenize)

In [13]:
freq_tweet = vectorize.fit_transform(train)
freq_tweet.shape

(23750, 56333)

In [14]:
classe = classe.replace({0:'negativo',1:'positivo',2:'neutro'})


In [15]:
modelo = MultinomialNB()
modelo.fit(freq_tweet,classe)

MultinomialNB()

In [16]:
def marque_negacao(texto):
    negacoes = ['não','nem','negativo','nenhum','nada','nunca','jamais']
    negacao_detectada = False
    resultado = []
    palavras = texto.split()
    for p in palavras:
        p = p.lower()
        if negacao_detectada == True:
            p = p + '_NEG'
        if p in negacoes:
            negacao_detectada = True
        resultado.append(p)
    return (" ".join(resultado))

In [17]:
from sklearn.pipeline import Pipeline

In [18]:
pipeline_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', MultinomialNB())
])

In [19]:
pipeline_negacoes = Pipeline([
  ('counts', CountVectorizer(tokenizer=lambda text: marque_negacao(text))),
  ('classifier', MultinomialNB())
])

In [20]:
pipeline_simples.fit(train,classe)

Pipeline(steps=[('counts', CountVectorizer()), ('classifier', MultinomialNB())])

In [21]:
resultados = cross_val_predict(pipeline_simples, train, classe, cv=10)

In [22]:
pipeline_svm_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', svm.SVC(kernel='linear'))
])

In [23]:
pipeline_svm_negacoes = Pipeline([
  ('counts', CountVectorizer(tokenizer=lambda text: marque_negacao(text))),
  ('classifier', svm.SVC(kernel='linear'))
])

In [24]:
def Metricas(modelo, tweets, classes):
  resultados = cross_val_predict(modelo, tweets, classes, cv=10)
  return 'Acurácia do modelo: {}'.format(metrics.accuracy_score(classes,resultados))

In [25]:
Metricas(pipeline_simples,train,classe)

'Acurácia do modelo: 0.7810526315789473'

In [27]:
Metricas(pipeline_negacoes,train,classe)

'Acurácia do modelo: 0.8088421052631579'

In [ ]:
Metricas(pipeline_svm_simples,train,classe)

In [ ]:
Metricas(pipeline_svm_negacoes,train,classe)

In [26]:
negative = trainer.loc[trainer['sentiment']==0]
negative = negative['tweet_text']

In [ ]:
vocabulario = []
for frase in negative:
    for palavra in frase.split():
        
        #não queremos palavras de uma única letra (pode acontecer devido ao stemming...)
        if len(palavra) > 1:
            if palavra not in [x[0] for x in vocabulario]:
                vocabulario.append([palavra, 1])
            else:
                vocabulario[[x[0] for x in vocabulario].index(palavra)][1] += 1
            
print("\nO vocabulário é formado por N =", len(vocabulario), "palavras!")

#a partir do vocabulário, crio um dataframe com a contagem
vocab_count = pd.DataFrame({"palavra": [],
                            "count": []})

vocab_count["palavra"] = pd.Series(vocabulario).apply(lambda x: x[0])
vocab_count["count"] = pd.Series(vocabulario).apply(lambda x: x[1])
vocab_count = vocab_count.sort_values("count", ascending=False)

print("\nTemos a seguir as 10 mais comuns, com as respectivas contagens:")
display(vocab_count.head(10))

In [ ]:
CONCLUSAO
foi feito um pre-processamento padrao nos textos com e sem o steemer
(as vezes nao funciona mto bem no portugues) utlizei o tokenizer do twiter 
para que os emojis , hastags e @usuario nao fosse excluidos.
utilizei neivebayes e svm como modelo e nos dois apliquei uma funcao de negacao 
para que o modelo dese mais relevancia as palavras de negação...